# "March Madness 2020"
> "Predict the winner of the Mens NCAA March Madness Event"
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

This tournament was cancelled due to COVID-19 so I have stopped development

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [50]:
sub = pd.read_csv('./MSampleSubmissionStage1_2020.csv')

In [51]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11390 entries, 0 to 11389
Data columns (total 2 columns):
ID      11390 non-null object
Pred    11390 non-null float64
dtypes: float64(1), object(1)
memory usage: 133.5+ KB


In [52]:
sub['Season'] = sub['ID'].str[:4]
sub['LowTeamID'] = sub['ID'].str[5:9]
sub['HighTeamID'] = sub['ID'].str[10:]

In [53]:
sub.head()

,ID,Pred,Season,LowTeamID,HighTeamID
0,2015_1107_1112,0.5,2015,1107,1112
1,2015_1107_1116,0.5,2015,1107,1116
2,2015_1107_1124,0.5,2015,1107,1124
3,2015_1107_1125,0.5,2015,1107,1125
4,2015_1107_1129,0.5,2015,1107,1129


## Regular Season Data

In [54]:
RSR = pd.read_csv('./MRegularSeasonCompactResults.csv')

In [55]:
RSR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161552 entries, 0 to 161551
Data columns (total 8 columns):
Season     161552 non-null int64
DayNum     161552 non-null int64
WTeamID    161552 non-null int64
WScore     161552 non-null int64
LTeamID    161552 non-null int64
LScore     161552 non-null int64
WLoc       161552 non-null object
NumOT      161552 non-null int64
dtypes: int64(7), object(1)
memory usage: 9.2+ MB


In [56]:
RSR = RSR[RSR['Season']>=2015]

In [57]:
RSR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26986 entries, 134566 to 161551
Data columns (total 8 columns):
Season     26986 non-null int64
DayNum     26986 non-null int64
WTeamID    26986 non-null int64
WScore     26986 non-null int64
LTeamID    26986 non-null int64
LScore     26986 non-null int64
WLoc       26986 non-null object
NumOT      26986 non-null int64
dtypes: int64(7), object(1)
memory usage: 1.8+ MB


In [58]:
SeasonWins = RSR.groupby(['Season','WTeamID'],as_index=False)['DayNum'].count().rename(columns={'DayNum':'Wins'})
SeasonLosses = RSR.groupby(['Season','LTeamID'],as_index=False)['DayNum'].count().rename(columns={'DayNum':'Losses'})

In [59]:
SeasonWins.head()

,Season,WTeamID,Wins
0,2015,1101,7
1,2015,1102,12
2,2015,1103,20
3,2015,1104,17
4,2015,1105,8


In [60]:
SeasonLosses.head()

,Season,LTeamID,Losses
0,2015,1101,21
1,2015,1102,17
2,2015,1103,14
3,2015,1104,14
4,2015,1105,20


In [61]:
SeasonRecord = pd.merge(SeasonWins,SeasonLosses,left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])

In [62]:
SeasonRecord.head()

,Season,WTeamID,Wins,LTeamID,Losses
0,2015,1101,7,1101,21
1,2015,1102,12,1102,17
2,2015,1103,20,1103,14
3,2015,1104,17,1104,14
4,2015,1105,8,1105,20


In [63]:
SeasonRecord['Pct'] = SeasonRecord['Wins']/(SeasonRecord['Wins']+SeasonRecord['Losses'])

In [64]:
SeasonRecord.head()

,Season,WTeamID,Wins,LTeamID,Losses,Pct
0,2015,1101,7,1101,21,0.250000
1,2015,1102,12,1102,17,0.413793
2,2015,1103,20,1103,14,0.588235
3,2015,1104,17,1104,14,0.548387
4,2015,1105,8,1105,20,0.285714


## Post Season Data

In [65]:
PSR = pd.read_csv('./MNCAATourneyCompactResults.csv')

In [66]:
PSR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251 entries, 0 to 2250
Data columns (total 8 columns):
Season     2251 non-null int64
DayNum     2251 non-null int64
WTeamID    2251 non-null int64
WScore     2251 non-null int64
LTeamID    2251 non-null int64
LScore     2251 non-null int64
WLoc       2251 non-null object
NumOT      2251 non-null int64
dtypes: int64(7), object(1)
memory usage: 131.9+ KB


In [67]:
PSR.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [68]:
PSR = PSR[PSR['Season']>=2015]
PSR['LowTeamID'] = np.where(PSR['LTeamID']<PSR['WTeamID'],PSR['LTeamID'],PSR['WTeamID'])
PSR['HighTeamID'] = np.where(PSR['LTeamID']<PSR['WTeamID'],PSR['WTeamID'],PSR['LTeamID'])

In [69]:
PSR.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,LowTeamID,HighTeamID
1916,2015,134,1214,74,1264,64,N,0,1214,1264
1917,2015,134,1279,94,1140,90,N,0,1140,1279
1918,2015,135,1173,56,1129,55,N,0,1129,1173
1919,2015,135,1352,81,1316,77,N,0,1316,1352
1920,2015,136,1112,93,1411,72,N,0,1112,1411


In [70]:
PSR['LowTeamWin'] = np.where(PSR['LowTeamID']==PSR['WTeamID'],1,0)

In [71]:
PSR.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,LowTeamID,HighTeamID,LowTeamWin
1916,2015,134,1214,74,1264,64,N,0,1214,1264,1
1917,2015,134,1279,94,1140,90,N,0,1140,1279,0
1918,2015,135,1173,56,1129,55,N,0,1129,1173,0
1919,2015,135,1352,81,1316,77,N,0,1316,1352,0
1920,2015,136,1112,93,1411,72,N,0,1112,1411,1


## Features Data

In [72]:
df = pd.DataFrame(PSR[['Season','LowTeamID','HighTeamID','LowTeamWin']])

In [73]:
df = pd.merge(df,SeasonRecord[['Season','LTeamID','Pct']],how='left',left_on=['Season','LowTeamID'],right_on=['Season','LTeamID'])

In [74]:
df = pd.merge(df,SeasonRecord[['Season','LTeamID','Pct']],how='left',left_on=['Season','HighTeamID'],right_on=['Season','LTeamID'],suffixes=('_low','_high'))

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335 entries, 0 to 334
Data columns (total 8 columns):
Season          335 non-null int64
LowTeamID       335 non-null int64
HighTeamID      335 non-null int64
LowTeamWin      335 non-null int32
LTeamID_low     332 non-null float64
Pct_low         332 non-null float64
LTeamID_high    333 non-null float64
Pct_high        333 non-null float64
dtypes: float64(4), int32(1), int64(3)
memory usage: 22.2 KB


In [76]:
df.head()

,Season,LowTeamID,HighTeamID,LowTeamWin,LTeamID_low,Pct_low,LTeamID_high,Pct_high
0,2015,1214,1264,1,1214.0,0.484848,1264.0,0.593750
1,2015,1140,1279,0,1140.0,0.718750,1279.0,0.625000
2,2015,1129,1173,0,1129.0,0.741935,1173.0,0.757576
3,2015,1316,1352,0,1316.0,0.645161,1352.0,0.575758
4,2015,1112,1411,1,1112.0,0.911765,1411.0,0.647059


In [83]:
df['Pct_diff'] = df['Pct_low'] - df['Pct_high']
df = df.dropna()

## Model build

In [97]:
x = df[['Pct_diff']]
y = df['LowTeamWin']
X_train, X_test, y_train, y_test = train_test_split(x,y,random_state=0)

In [98]:
clf = LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_train,y_train)

0.5991902834008097

In [99]:
clf.score(X_test,y_test)

0.6265060240963856

In [100]:
clf.coef_

array([[2.331166]])